# Multi-agent collaboration Inline : Evidence-Based NSCLC Target Discovery Hypothesis


**Hypothesis Generation Agent** responsible for formulating testable scientific hypotheses about how the target genes may influence cancer biology and treatment response. 
 This agent will use two existing agents - 

    - Biological Pathways agent for analyzing molecular mechanisms and pathway contexts of potential cancer targets.
    - Evidence Researcher Agent responsible for literature evidence research from PubMed and internal papers in a knowledge base

# !pip3 install -r requirements.txt
!pip3 install --upgrade boto3
!pip3 show boto3

In [1]:
!pip freeze | grep boto3

boto3==1.37.33


In [2]:
import boto3

In [3]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

# FM used for all agents, choose cross-region if needed
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
# agent_foundation_model = ["us.anthropic.claude-3-5-sonnet-20241022-v2:0"]

# Supervisor agent FM, choose cross-region if needed
supervisor_agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
# supervisor_agent_foundation_model = ["us.anthropic.claude-3-5-sonnet-20241022-v2:0"]

In [4]:
region = session.region_name
region

'us-west-2'

In [ ]:
# Retrieve the information from the Bedrock Agents console based on their names and fill in below
from boto3 import client
from botocore.config import Config

config = Config(read_timeout=2000)

bedrock_agent_client = boto3.client(service_name='bedrock-agent', 
                      region_name=region,
                      config=config)
#bedrock_agent_client = boto3.client('bedrock-agent', region)

# Biological-pathways-analyst
pathways_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='XXXX',
    agentId='XXXX'
)['agentAlias']['agentAliasArn']

# Clinical-evidence-researcher
research_evidence_agent_alias_arn = bedrock_agent_client.get_agent_alias(
    agentAliasId='XXXX',
    agentId='XXXX'
)['agentAlias']['agentAliasArn']

pathways_agent_alias_arn, research_evidence_agent_alias_arn

In [6]:
import os
import json
from pprint import pprint
import boto3
from datetime import datetime
import random
import pprint
from termcolor import colored
from rich.console import Console
from rich.markdown import Markdown
import uuid

session = boto3.session.Session()
region = session.region_name
config = Config(read_timeout=2000)
# Runtime Endpoints
bedrock_rt_client = boto3.client(
    "bedrock-agent-runtime",
    region_name=region, config=config
)

def invoke_inline_agent_helper(client, request_params, trace_level="core"):
    _time_before_call = datetime.now()

    _agent_resp = client.invoke_inline_agent(
        **request_params
    )

    if request_params["enableTrace"]:
        if trace_level == "all":
            print(f"invokeAgent API response object: {_agent_resp}")
        else:
            print(
                f"invokeAgent API request ID: {_agent_resp['ResponseMetadata']['RequestId']}"
            )
            session_id = request_params["sessionId"]
            print(f"invokeAgent API session ID: {session_id}")

    # Return error message if invoke was unsuccessful
    if _agent_resp["ResponseMetadata"]["HTTPStatusCode"] != 200:
        _error_message = f"API Response was not 200: {_agent_resp}"
        if request_params["enableTrace"] and trace_level == "all":
            print(_error_message)
        return _error_message

    _total_in_tokens = 0
    _total_out_tokens = 0
    _total_llm_calls = 0
    _orch_step = 0
    _sub_step = 0
    _trace_truncation_lenght = 300
    _time_before_orchestration = datetime.now()

    _agent_answer = ""
    _event_stream = _agent_resp["completion"]

    try:
        for _event in _event_stream:
            _sub_agent_alias_id = None

            if "chunk" in _event:
                _data = _event["chunk"]["bytes"]
                _agent_answer = _data.decode("utf8")

            if "trace" in _event and request_params["enableTrace"]:
                if "failureTrace" in _event["trace"]["trace"]:
                    print(
                        colored(
                            f"Agent error: {_event['trace']['trace']['failureTrace']['failureReason']}",
                            "red",
                        )
                    )

                if "orchestrationTrace" in _event["trace"]["trace"]:
                    _orch = _event["trace"]["trace"]["orchestrationTrace"]

                    if trace_level in ["core", "outline"]:
                        if "rationale" in _orch:
                            _rationale = _orch["rationale"]
                            print(colored(f"{_rationale['text']}", "blue"))

                        if "invocationInput" in _orch:
                            # NOTE: when agent determines invocations should happen in parallel
                            # the trace objects for invocation input still come back one at a time.
                            _input = _orch["invocationInput"]
                            print(_input)

                            if "actionGroupInvocationInput" in _input:
                                if 'function' in _input['actionGroupInvocationInput']:
                                    tool = _input['actionGroupInvocationInput']['function']
                                elif 'apiPath' in _input['actionGroupInvocationInput']:
                                    tool = _input['actionGroupInvocationInput']['apiPath']
                                else:
                                    tool = 'undefined'
                                if trace_level == "outline":
                                    print(
                                        colored(
                                            f"Using tool: {tool}",
                                            "magenta",
                                        )
                                    )
                                else:
                                    print(
                                        colored(
                                            f"Using tool: {tool} with these inputs:",
                                            "magenta",
                                        )
                                    )
                                    if (
                                        'parameters' in _input['actionGroupInvocationInput']
                                    ) and (
                                        len(
                                            _input["actionGroupInvocationInput"][
                                                "parameters"
                                            ]
                                        )
                                        == 1
                                    ) and (
                                        _input["actionGroupInvocationInput"][
                                            "parameters"
                                        ][0]["name"]
                                        == "input_text"
                                    ):
                                        print(
                                            colored(
                                                f"{_input['actionGroupInvocationInput']['parameters'][0]['value']}",
                                                "magenta",
                                            )
                                        )
                                    else:
                                        print(
                                            colored(
                                                f"{_input['actionGroupInvocationInput']}\n",
                                                "magenta",
                                            )
                                        )

                            elif "codeInterpreterInvocationInput" in _input:
                                if trace_level == "outline":
                                    print(
                                        colored(
                                            f"Using code interpreter", "magenta"
                                        )
                                    )
                                else:
                                    console = Console()
                                    _gen_code = _input[
                                        "codeInterpreterInvocationInput"
                                    ]["code"]
                                    _code = f"```python\n{_gen_code}\n```"

                                    console.print(
                                        Markdown(f"**Generated code**\n{_code}")
                                    )

                        if "observation" in _orch:
                            if trace_level == "core":
                                _output = _orch["observation"]
                                if "actionGroupInvocationOutput" in _output:
                                    print(
                                        colored(
                                            f"--tool outputs:\n{_output['actionGroupInvocationOutput']['text'][0:_trace_truncation_lenght]}...\n",
                                            "magenta",
                                        )
                                    )

                                if "agentCollaboratorInvocationOutput" in _output:
                                    _collab_name = _output[
                                        "agentCollaboratorInvocationOutput"
                                    ]["agentCollaboratorName"]
                                    _collab_output_text = _output[
                                        "agentCollaboratorInvocationOutput"
                                    ]["output"]["text"][0:_trace_truncation_lenght]
                                    print(
                                        colored(
                                            f"\n----sub-agent {_collab_name} output text:\n{_collab_output_text}...\n",
                                            "magenta",
                                        )
                                    )

                                if "finalResponse" in _output:
                                    print(
                                        colored(
                                            f"Final response:\n{_output['finalResponse']['text'][0:_trace_truncation_lenght]}...",
                                            "cyan",
                                        )
                                    )


                    if "modelInvocationOutput" in _orch:
                        _orch_step += 1
                        _sub_step = 0
                        print(colored(f"---- Step {_orch_step} ----", "green"))

                        _llm_usage = _orch["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = 0 
                        if ('inputTokens' in _llm_usage):
                            _in_tokens = _llm_usage["inputTokens"]
                        _total_in_tokens += _in_tokens
                        _out_tokens = 0
                        if ('outputTokens' in _llm_usage):
                            _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1
                        _orch_duration = (
                            datetime.now() - _time_before_orchestration
                        )

                        print(
                            colored(
                                f"Took {_orch_duration.total_seconds():,.1f}s, using {_in_tokens+_out_tokens} tokens (in: {_in_tokens}, out: {_out_tokens}) to complete prior action, observe, orchestrate.",
                                "yellow",
                            )
                        )

                        # restart the clock for next step/sub-step
                        _time_before_orchestration = datetime.now()

                elif "preProcessingTrace" in _event["trace"]["trace"]:
                    _pre = _event["trace"]["trace"]["preProcessingTrace"]
                    if "modelInvocationOutput" in _pre:
                        _llm_usage = _pre["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = 0 
                        if ('inputTokens' in _llm_usage):
                            _in_tokens = _llm_usage["inputTokens"]
                        _total_in_tokens += _in_tokens
                        _out_tokens = 0
                        if ('outputTokens' in _llm_usage):
                            _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1

                        print(
                            colored(
                                "Pre-processing trace, agent came up with an initial plan.",
                                "yellow",
                            )
                        )
                        print(
                            colored(
                                f"Used LLM tokens, in: {_in_tokens}, out: {_out_tokens}",
                                "yellow",
                            )
                        )

                elif "postProcessingTrace" in _event["trace"]["trace"]:
                    _post = _event["trace"]["trace"]["postProcessingTrace"]
                    if "modelInvocationOutput" in _post:
                        _llm_usage = _post["modelInvocationOutput"]["metadata"][
                            "usage"
                        ]
                        _in_tokens = 0 
                        if ('inputTokens' in _llm_usage):
                            _in_tokens = _llm_usage["inputTokens"]
                        _total_in_tokens += _in_tokens
                        _out_tokens = 0
                        if ('outputTokens' in _llm_usage):
                            _out_tokens = _llm_usage["outputTokens"]
                        _total_out_tokens += _out_tokens

                        _total_llm_calls += 1
                        print(colored("Agent post-processing complete.", "yellow"))
                        print(
                            colored(
                                f"Used LLM tokens, in: {_in_tokens}, out: {_out_tokens}",
                                "yellow",
                            )
                        )

                if trace_level == "all":
                    print(json.dumps(_event["trace"], indent=2))

            if "files" in _event.keys() and request_params["enableTrace"]:
                console = Console()
                files_event = _event["files"]
                console.print(Markdown("**Files**"))

                files_list = files_event["files"]
                for this_file in files_list:
                    print(f"{this_file['name']} ({this_file['type']})")
                    file_bytes = this_file["bytes"]

                    # save bytes to file, given the name of file and the bytes
                    file_name = os.path.join("output", this_file["name"])
                    with open(file_name, "wb") as f:
                        f.write(file_bytes)

        if request_params["enableTrace"]:
            duration = datetime.now() - _time_before_call

            if trace_level in ["core", "outline"]:
                print(
                    colored(
                        f"Agent made a total of {_total_llm_calls} LLM calls, "
                        + f"using {_total_in_tokens+_total_out_tokens} tokens "
                        + f"(in: {_total_in_tokens}, out: {_total_out_tokens})"
                        + f", and took {duration.total_seconds():,.1f} total seconds",
                        "yellow",
                    )
                )

            if trace_level == "all":
                print(f"Returning agent answer as: {_agent_answer}")

        return _agent_answer

    except Exception as e:
        print(f"Caught exception while processing input to invokeAgent:\n")
        input_text = request_params["inputText"]
        print(f"  for input text:\n{input_text}\n")
        print(
            f"  request ID: {_agent_resp['ResponseMetadata']['RequestId']}, retries: {_agent_resp['ResponseMetadata']['RetryAttempts']}\n"
        )
        print(f"Error: {e}")
        raise Exception("Unexpected exception: ", e)

In [7]:
pathways_agent_alias_arn

'arn:aws:bedrock:us-west-2:048051882663:agent-alias/XNKNELHZ6U/AGC98M3UQL'

## Setup the supervisor agent for hypothesis generation inline

In [12]:
sub_agents_list = [
    {
        'agentAliasArn': pathways_agent_alias_arn,
        'collaboratorInstruction': """Use this agent specialized in working with biological pathways and molecular mechanisms of poteintial cancer targets using Reactome data . Its primary task is to interpret user queries, generate and execute appropriate open Cypher queries.""",
        'collaboratorName': 'ReactomePathwaysAnalyst',
        'relayConversationHistory': 'TO_COLLABORATOR'
    },
    {
        'agentAliasArn': research_evidence_agent_alias_arn,
        'collaboratorInstruction': """Use this agent specialized in summarizing internal and external evidence related to cancer biomarkers. Its primary task is to interpret user queries, gather internal and external evidence, and provide relevant medical insights based on the results.""",
        'collaboratorName': 'ClincialEvidenceResearcher',
        'relayConversationHistory': 'TO_COLLABORATOR'
    }
]
agent_name = "multi-agent-hypothesis-generator"
agent_description = "Multi-agent collaboration for hypothesis generation for Target discovery"
agent_instruction = """You are the Hypothesis Generation Agent responsible for formulating testable scientific hypotheses about how the target genes may influence cancer biology and treatment response. 
You create mechanistically sound hypotheses based on available evidence.
Your primary task is to interpret user goal to generate a hypothesis for Target discovery for non small cell lung cancer , use relevant agents for specific tasks, and provide consolidated 
new hypothesis based on the data and out of the box thinking. You can provide responses from a prior agent to the next agent in sequence or in parallel. 

1. BiologicalPathwaysAnalyst
- Executes Cypher queries for reactome data retrieval
- Processes biological pathways and mechanistic data 
- Check protein interactions and immune relevance
- For immune relevance query Reactome for immune pathway connections:" 
MATCH (g:ReferenceGeneProduct)-[:referenceEntity]-()-[:input|output|catalystActivity|physicalEntity*]->(p:Pathway)
WHERE g.displayName = '{gene_symbol}' AND p.displayName CONTAINS 'immune'
RETURN p.displayName as ImmunePathway"
2. ClinicalEvidenceResearcher
- Synthesizes internal and external research evidence
- Provides context from existing medical literature
- Summarizes relevant clinical findings

When generating a hypothesis for non small cell lung cancer:
1. First, collect literature evidence using the Clinical Evidence Researcher:
    * Call clinical_evidence_researcher to get evidence
2. Get mechanism data from the Enhanced Biological Pathways Agent:
    * Call biological_pathways_agent to query the target gene, check protein interactions and immune relevance
3. Analyze radiogenomic context if available:
    * Extract correlations between the target and radiomic features
    * Focus on features related to tumor heterogeneity, immune infiltration, or vascularity
4. Formulate a detailed, original, and specific single hypothesis for achieving the stated goal, leveraging ideas from literature. This should not be a mere aggregation of literature. Think out-of-the-box with this structure:
    * Intervention: What happens when the target is inhibited/activated?
    * Mechanism: Through which pathway(s) does the intervention work?
    * Context: In which tumor types/subtypes will this be most effective?
    * Outcome: What is the expected clinical or biological result?
5. Consider the tumor context in your hypothesis:
    * For "cold" tumors, focus on enhancing T cell infiltration
    * For "hot" tumors, focus on enhancing T cell function
    * Include TMB status: "high" TMB may suggest better response to immunotherapy approaches
6. Reflect on the generated hypothesis and suggest improvements based on that reflection and understanding of literature evidence. Your task is to analyze the
relationship between a provided hypothesis and observations from a scientific article. Specifically, determine if the hypothesis provides a novel causal explanation
for the observations, or if they contradict it. If necessary, suggest to create or modify the hypothesis again iteratively
7. Produce and save a markdown-formatted report based on the final hypothesis structure. As a Report Writer you are excellent at taking input and 
    producing a cleanly formatted and easily readable report, saved in Markdown format in the agent store. 
    1/ Provide a concise introduction to the relevant scientific domain.
    2/. Summarize recent findings and pertinent research, highlighting successful approaches.
    3/. Identify promising avenues for exploration that may yield innovative hypotheses.
    4/ Core hypothesis : Compile a detailed, innovative, and technologically viable hypothesis to achieve the objective, emphasizing simplicity and practicality.
    To simplify the task, break it into small sub-tasks, each requiring only specific 
    input and only generating a nicely formatted version of each specific section. 
    Save each section to its own key in the agent store. Once all sub-sections have been formatted
    and saved, retrieve each section from the agent store one last time, concatenate, and save
    the final report to the agent store with the entire text, not just placeholders.

Make sure you perform Steps 1 to 7 above to produce a final hypothesis report. 
"""
#agent_collaboration = 'SUPERVISOR_ROUTER'
agent_collaboration = 'SUPERVISOR'
enable_trace = True
end_session = False
input_text = """Can you generate hypotheses for novel targets for non small cell lung cancer that might prevent antigen escape resistance mechanism?"""
foundation_model = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'   
session_id = str(uuid.uuid1())

request_params = {
    "instruction": agent_instruction,
    "foundationModel": foundation_model,
    "sessionId": session_id,
    "endSession": end_session,
    "enableTrace": enable_trace,
    "agentCollaboration": agent_collaboration,
    "inputText": input_text
}
request_params['collaboratorConfigurations'] = sub_agents_list

In [13]:
answer = invoke_inline_agent_helper(bedrock_rt_client, request_params, trace_level="core")

invokeAgent API request ID: 79c43af0-9809-4154-af18-7cf53d310ea7
invokeAgent API session ID: 8ce904f4-1890-11f0-b46d-76e6d89ee90f
---- Step 1 ----
Took 5.3s, using 2197 tokens (in: 1995, out: 202) to complete prior action, observe, orchestrate.
I'll follow the structured approach to generate hypotheses for novel targets in NSCLC focusing on antigen escape resistance mechanisms. First, I'll gather clinical evidence and pathway information in parallel to build a comprehensive understanding.
{'agentCollaboratorInvocationInput': {'agentCollaboratorAliasArn': 'arn:aws:bedrock:us-west-2:048051882663:agent-alias/UJUO93AYZU/MXQDS9TIZF', 'agentCollaboratorName': 'ClincialEvidenceResearcher', 'input': {'text': 'Please provide a comprehensive summary of recent clinical evidence regarding antigen escape mechanisms in non-small cell lung cancer, focusing on:\n1. Known mechanisms of antigen escape\n2. Current therapeutic approaches targeting these mechanisms\n3. Biomarkers associated with antigen es

In [14]:
answer

'# Hypothesis Report: Novel Target Strategy for Preventing Antigen Escape in NSCLC\n\n## Introduction\nNon-small cell lung cancer (NSCLC) frequently develops resistance to immunotherapy through antigen escape mechanisms, limiting the long-term efficacy of current treatments. Recent evidence suggests that defective antigen presentation and processing are key contributors to this resistance.\n\n## Recent Findings Summary\n- Only 17% of NSCLC cases show favorable T2 (TIL+, PD-L1+) phenotype\n- Multiple escape mechanisms operate simultaneously\n- Antigen presentation machinery defects are common in resistant tumors\n- Cross-presentation pathway components show therapeutic potential\n\n## Core Hypothesis\n\n### Novel Target: TRIM23-MHC-I Axis Enhancement Strategy\n\n**Intervention:**\nDual targeting of TRIM23 (Tripartite Motif Containing 23) ubiquitin ligase activation and stabilization of MHC class I complex through small molecule enhancement.\n\n**Mechanism:**\n1. TRIM23 activation enhanc

## Next Steps
Congratulations! We've now created a supervisor agent at runtime for hypothesis generation.